## Scrapy之LinkExtractor、Rule链接提取
　　参考：[Scrapy框架学习（四）----CrawlSpider、LinkExtractors、Rule及爬虫示](https://blog.csdn.net/qq_33689414/article/details/78669514)<br>
　　　　　[Scrapy 动态配置爬虫 LinkExtractor提取链接](https://blog.csdn.net/fan13938409755/article/details/105089248/)<br>
  
### 一、LinkExtractor
LxmlLinkExtractor 是一种强大的链接提取器，使用他能很方便的进行从网页(scrapy.http.Response )中抽取会被follow的链接的对象，他是通过xml中强大的HTMLParser实现的。
每个LinkExtractor有唯一的公共方法是 extract_links()，它接收一个 Response 对象，并返回一个 scrapy.link.Link 对象。
Link Extractors要实例化一次，并且 extract_links 方法会根据不同的 response 调用多次提取链接｡

In [ ]:
# from .lxmlhtml import LxmlLinkExtractor as LinkExtractor

class LxmlLinkExtractor(FilteringLinkExtractor):

    def __init__(self, allow=(), deny=(), allow_domains=(), deny_domains=(), restrict_xpaths=(),
                 tags=('a', 'area'), attrs=('href',), canonicalize=False,
                 unique=True, process_value=None, deny_extensions=None, restrict_css=(),
                 strip=True, restrict_text=None):
        tags, attrs = set(arg_to_iter(tags)), set(arg_to_iter(attrs))
        tag_func = lambda x: x in tags
        attr_func = lambda x: x in attrs
        lx = LxmlParserLinkExtractor(
            tag=tag_func,
            attr=attr_func,
            unique=unique,
            process=process_value,
            strip=strip,
            canonicalized=canonicalize
        )

        super(LxmlLinkExtractor, self).__init__(lx, allow=allow, deny=deny,
                                                allow_domains=allow_domains, deny_domains=deny_domains,
                                                restrict_xpaths=restrict_xpaths, restrict_css=restrict_css,
                                                canonicalize=canonicalize, deny_extensions=deny_extensions,
                                                restrict_text=restrict_text)

    def extract_links(self, response):
        base_url = get_base_url(response)
        if self.restrict_xpaths:
            docs = [subdoc
                    for x in self.restrict_xpaths
                    for subdoc in response.xpath(x)]
        else:
            docs = [response.selector]
        all_links = []
        for doc in docs:
            links = self._extract_links(doc, response.url, response.encoding, base_url)
            all_links.extend(self._process_links(links))
        return unique_list(all_links)

参数说明：
>allow=(一个正则表达式或者正则表达式的列表) 只有与之相匹配的url才能被提取出来
deny=(一个正则表达式或者正则表达式的列表) 一个正则表达式（或正则表达式列表），（绝对）urls必须匹配才能排除（即不提取）。它优先于allow参数。如果没有给出（或为空），它不会排除任何链接。
allow_domains=(str或者list) 允许提取链接的域名的字符串列表或者单个字符串，例如：allow_domain = \['baidu.com'\]则只能提取baidu.com的域名内的链接
deny_domains=() 与上述的意思刚刚相反
restrict_xpaths=(str或list) - 是一个XPath（或XPath的列表），它定义响应中应从中提取链接的区域。如果给出，只有那些XPath选择的文本将被扫描链接。
targs=('a','area') 标签或在提取链接时要考虑的标签列表。默认为。('a', 'area') 也就是默认只有a标签与area标签的链接才能被提取
attrs=('href',) 在查找要提取的链接时应该考虑的属性或属性列表（仅适用于参数中指定的那些标签tags ）。默认为('href',)
cononicalize=(boolean) 规范化每个提取的url（使用w3lib.url.canonicalize_url）。默认为True。
unique=(boolean) 是否应对提取的链接应用重复过滤。
process_value=(callable) 接收从标签提取的每个值和扫描的属性并且可以修改值并返回新值的函数，或者返回None以完全忽略链接。如果没有给出，那么process_value默认为:lambda x:x 。用法示例如下：
要从此代码中提取链接：\<a href="javascript\:goToPage('../other/page.html'); return false">Link text</a>您可以使用以下功能process_value：
def process_value(value):
    m = re.search("javascript\:goToPage\('(.*?)'", value)
    if m:
        return m.group(1)
deny_extensions=(list) -包含在提取链接时应该忽略的扩展的单个值或字符串列表。如果没有给出，它将默认为IGNORED_EXTENSIONS在scrapy.linkextractors包中定义的 列表 。
restrict_css=() 一个CSS选择器（或选择器列表），用于定义响应中应提取链接的区域。有相同的行为restrict_xpaths。
strip=True 这个是把地址前后多余的空格删除，很有必要

### 二、Rule
在rules中包含一个或多个Rule对象，每个Rule对爬取网站的动作定义了特定操作。如果多个rule匹配了相同的链接，则根据规则在本集合中被定义的顺序，第一个会被使用。


In [ ]:
def _identity(request, response):
    return request
def _get_method(method, spider):
    if callable(method):
        return method
    elif isinstance(method, six.string_types):
        return getattr(spider, method, None)

class Rule(object):

    def __init__(self, link_extractor=None, callback=None, cb_kwargs=None, follow=None, process_links=None, process_request=None):
        self.link_extractor = link_extractor or _default_link_extractor
        self.callback = callback
        self.cb_kwargs = cb_kwargs or {}
        self.process_links = process_links
        self.process_request = process_request or _identity
        self.process_request_argcount = None
        self.follow = follow if follow is not None else not callback

    def _compile(self, spider):
        self.callback = _get_method(self.callback, spider)
        self.process_links = _get_method(self.process_links, spider)
        self.process_request = _get_method(self.process_request, spider)
        self.process_request_argcount = len(get_func_args(self.process_request))
        if self.process_request_argcount == 1:
            msg = 'Rule.process_request should accept two arguments (request, response), accepting only one is deprecated'
            warnings.warn(msg, category=ScrapyDeprecationWarning, stacklevel=2)

    def _process_request(self, request, response):
        """
        Wrapper around the request processing function to maintain backward
        compatibility with functions that do not take a Response object
        """
        args = [request] if self.process_request_argcount == 1 else [request, response]
        return self.process_request(*args)

参数说明：
>link_extractor：是一个Link Extractor对象。其定义了如何从爬取到的页面提取链接。
callback：是一个callable或string（该Spider中同名的函数将会被调用）。从link_extractor中每获取到链接时将会调用该函数。该回调函数接收一个response作为其第一个参数，并返回一个包含Item以及Request对象(或者这两者的子类)的列表。
cb_kwargs：包含传递给回调函数的参数（keyword argument）的字典。
follow：是一个boolean值，指定了根据该规则从response提取的链接是否需要跟进。如果callback为None，follow默认设置True，否则默认False。
当follow为True时，爬虫会从获取的response中取出符合规则的url，再次进行爬取，如果这次爬取的response中还存在符合规则的url，则再次爬取，无限循环，直到不存在符合规则的url。当follow为False是，爬虫只从start_urls 的response中取出符合规则的url，并请求.
process_links：是一个callable或string（该Spider中同名的函数将会被调用）。从link_extrator中获取到链接列表时将会调用该函数。该方法主要是用来过滤。
process_request：是一个callable或string（该spider中同名的函数都将会被调用）。该规则提取到的每个request时都会调用该函数。该函数必须返回一个request或者None。用来过滤request。

注意：当编写爬虫规则时，避免使用parse作为回调函数。由于CrawlSpider使用parse方法来实现其逻辑，如果覆盖了 parse方法，crawl spider将会运行失败。